# 학습 데이터 준비

In [1]:
import numpy as np

x_data = np.array([2, 4, 6, 8, 10, 12, 14, 16, 18, 20]).reshape(10,1)   
t_data = np.array([0, 0, 0, 0,  0,  0,  1,  1,  1,  1]).reshape(10,1)

print("x_data.shape = ", x_data.shape, ", t_data.shape = ", t_data.shape)

x_data.shape =  (10, 1) , t_data.shape =  (10, 1)


# 가중치W, 바이어스b 정의 (임의의 값)

In [2]:
W = np.random.rand(1,1)   #0~1사이 난수값, 결과 값 내적 방지
b = np.random.rand(1)  
print("W = ", W, ", W.shape = ", W.shape, ", b = ", b, ", b.shape = ", b.shape)

W =  [[0.66237227]] , W.shape =  (1, 1) , b =  [0.71523983] , b.shape =  (1,)


# Sigmoid(Z), 손실함수(cross-entropy) 정의

In [3]:
def sigmoid(x):#output의 손실을 방지하기 위해
    return 1 / (1+np.exp(-x))

def loss_func(x, t): 
    
    delta = 1e-7    # log 무한대 발산 방지, y=0 or 1인 경우 log내부 값 무한대
    
    z = np.dot(x,W) + b #행렬곱
    y = sigmoid(z)
    
    # cross-entropy 
    return  -np.sum( t*np.log(y + delta) + (1-t)*np.log((1 - y)+delta ) ) 


## 수치미분 정의

In [4]:
def numerical_derivative(f,x): #x 임의의 행렬
    delta_x=1e-4
    grad=np.zeros_like(x) #계산된 수치미분 값 저장 변수
    it = np.nditer(x,flags=['multi_index'], op_flags=['readwrite'])
    #모든 입력 변수에 대해 편미분 하기위한 iterator
    #nider배열 반복해 요소 하나에 접근해 설정에 따라 값을 알려줌
    
    while not it.finished: #변수 개수 만큼
        idx=it.multi_index #행렬에서 가리키는 값
        
        tmp=x[idx] #numpy 타입== mutable (변할 수 있음)이므로 원래 값 보관
        x[idx]=float(tmp)+delta_x #미세하게 변화시킴
        fx1=f(x) #f(x+delta_x)
        
        x[idx]=tmp-delta_x
        fx2=f(x) #f(x-delta_x)
        grad[idx]=(fx1-fx2)/(2*delta_x)
        
        x[idx]=tmp
        it.iternext()  #하나의 변수에 대한 수치미분 계산, 다음 변수로 넘어감
        
    return grad

## 학습, 계산

In [5]:
# 입력변수 x, t : numpy type
def error_val(x, t):
    delta = 1e-7    # log 무한대 발산 방지    
    z = np.dot(x,W) + b # dot()nn *n 차원, 벡터*스칼라 사용X 권장 ->np.multiply
    y = sigmoid(z)
    
    # cross-entropy 
    return  -np.sum( t*np.log(y + delta) + (1-t)*np.log((1 - y)+delta ) ) 

# 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
# 입력변수 x : numpy type
def predict(x):
    
    z = np.dot(x,W) + b
    y = sigmoid(z)
    
    if y >= 0.5:
        result = 1  # True
    else:
        result = 0  # False
    
    return y, result

In [6]:
learning_rate = 1e-2  # 발산하는 경우(오버슈팅), 1e-3 ~ 1e-6 등으로 바꾸어서 실행

f = lambda x : loss_func(x_data,t_data)  # f(x) = loss_func(x_data, t_data)

print("Initial error value = ", error_val(x_data, t_data), "Initial W = ", W, "\n", ", b = ", b )

for step in  range(20001): # 우도함수에서 발생확률이 최대가 되도록 W,b 업데이트 
    
    W -= learning_rate * numerical_derivative(f, W)
    
    b -= learning_rate * numerical_derivative(f, b)
    
    if (step % 400== 0): #계속 돌릴수록 손실함수값 작아짐, 수렴하지 않음-오버피팅
        print("step = ", step, "error value = ", error_val(x_data, t_data), "W = ", W, ", b = ",b )

Initial error value =  32.27899221774089 Initial W =  [[0.66237227]] 
 , b =  [0.71523983]
step =  0 error value =  15.075751405847486 W =  [[0.24693545]] , b =  [0.66169575]
step =  400 error value =  3.1490261390372725 W =  [[0.43473305]] , b =  [-4.13167904]
step =  800 error value =  1.7810327848178746 W =  [[0.45403878]] , b =  [-5.65007809]
step =  1200 error value =  1.5163772972963316 W =  [[0.53121879]] , b =  [-6.67775596]
step =  1600 error value =  1.3513854678483368 W =  [[0.59238585]] , b =  [-7.48999316]
step =  2000 error value =  1.2351908269886909 W =  [[0.64384341]] , b =  [-8.17180651]
step =  2400 error value =  1.1472012463762413 W =  [[0.68870413]] , b =  [-8.76515759]
step =  2800 error value =  1.0772852845791887 W =  [[0.72875061]] , b =  [-9.29405663]
step =  3200 error value =  1.0197970677098782 W =  [[0.76510662]] , b =  [-9.77362343]
step =  3600 error value =  0.9713046164002582 W =  [[0.79853003]] , b =  [-10.21404348]
step =  4000 error value =  0.9295

In [7]:
(real_val, logical_val) = predict(3) 

print(real_val, logical_val)

[[4.26152522e-07]] 0


In [8]:
(real_val, logical_val) = predict(17) 

print(real_val, logical_val)

[[0.99756338]] 1


## 다중입력 (2개)

In [9]:
import numpy as np

x_data = np.array([[2,4],[4,11],[6,6],[8,5],[10,7],[12,16],[14,8],[16,3],[18,7]])  
t_data = np.array([0, 0, 0, 0, 1,  1,  1,  1,  1]).reshape(9,1)

print("x_data.shape = ", x_data.shape, ", t_data.shape = ", t_data.shape)

x_data.shape =  (9, 2) , t_data.shape =  (9, 1)


In [10]:
#가중치,바이어스 정의
W=np.random.rand(2,1)
b=np.random.rand(1)
print("W = ", W, ", W.shape = ", W.shape, ", b = ", b, ", b.shape = ", b.shape)

W =  [[0.35727442]
 [0.52220294]] , W.shape =  (2, 1) , b =  [0.461313] , b.shape =  (1,)


In [11]:
#손실함수 정의
def sigmoid(x):
    return 1 / (1+np.exp(-x))

def loss_func(x, t):
    
    delta = 1e-7    # log 무한대 발산 방지
    
    z = np.dot(x,W) + b #행렬곱
    y = sigmoid(z)
    
    # cross-entropy 
    return  -np.sum( t*np.log(y + delta) + (1-t)*np.log((1 - y)+delta ) ) 


In [12]:
#수치미분 정의
def numerical_derivative(f,x): #x 임의의 행렬
    delta_x=1e-4
    grad=np.zeros_like(x) #계산된 수치미분 값 저장 변수
    it = np.nditer(x,flags=['multi_index'], op_flags=['readwrite'])
    #모든 입력 변수에 대해 편미분 하기위한 iterator
    #nider배열 반복해 요소 하나에 접근해 설정에 따라 값을 알려줌
    
    while not it.finished: #변수 개수 만큼
        idx=it.multi_index #행렬에서 가리키는 값
        
        tmp=x[idx] #numpy 타입== mutable (변할 수 있음)이므로 원래 값 보관
        x[idx]=float(tmp)+delta_x #미세하게 변화시킴
        fx1=f(x) #f(x+delta_x)
        
        x[idx]=tmp-delta_x
        fx2=f(x) #f(x-delta_x)
        grad[idx]=(fx1-fx2)/(2*delta_x)
        
        x[idx]=tmp
        it.iternext()  #하나의 변수에 대한 수치미분 계산, 다음 변수로 넘어감
        
    return grad

In [13]:
# 손실함수 값 계산 함수
# 입력변수 x, t : numpy type
def error_val(x, t):
    delta = 1e-7    # log 무한대 발산 방지
    
    z = np.dot(x,W) + b # dot()nn *n 차원, 벡터*스칼라 사용X 권장 ->np.multiply
    y = sigmoid(z)
    
    # cross-entropy 
    return  -np.sum( t*np.log(y + delta) + (1-t)*np.log((1 - y)+delta ) ) 

# 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
# 입력변수 x : numpy type
def predict(x):
    
    z = np.dot(x,W) + b
    y = sigmoid(z)
    
    if y >= 0.5:
        result = 1  # True
    else:
        result = 0  # False
    
    return y, result

In [14]:
earning_rate = 1e-2  # 발산하는 경우, 1e-3 ~ 1e-6 등으로 바꾸어서 실행

f = lambda x : loss_func(x_data,t_data)  # f(x) = loss_func(x_data, t_data)

print("Initial error value = ", error_val(x_data, t_data), "Initial W = ", W, "\n", ", b = ", b )

for step in  range(20001):   #모든 파라미터 동시에 업데이트(벡터화 구현이 더 이상적)
    
    W -= learning_rate * numerical_derivative(f, W)
    
    b -= learning_rate * numerical_derivative(f, b)
    
    if (step % 400== 0): #계속 돌릴수록 손실함수값 작아짐
        print("step = ", step, "error value = ", error_val(x_data, t_data), "W = ", W, ", b = ",b )

Initial error value =  22.612571610205872 Initial W =  [[0.35727442]
 [0.52220294]] 
 , b =  [0.461313]
step =  0 error value =  12.066845420724032 W =  [[0.15857673]
 [0.26413231]] , b =  [0.42436905]
step =  400 error value =  2.2500793929971645 W =  [[ 0.42324972]
 [-0.08509422]] , b =  [-2.66753582]
step =  800 error value =  1.5821751671846302 W =  [[ 0.53835418]
 [-0.02547492]] , b =  [-4.29005138]
step =  1200 error value =  1.2750713118972703 W =  [[0.62503358]
 [0.00962095]] , b =  [-5.39454352]
step =  1600 error value =  1.095054365885111 W =  [[0.69508032]
 [0.03469101]] , b =  [-6.24105089]
step =  2000 error value =  0.97445930819223 W =  [[0.75416202]
 [0.05468897]] , b =  [-6.93410939]
step =  2400 error value =  0.8865878254402473 W =  [[0.80541277]
 [0.0718209 ]] , b =  [-7.52575009]
step =  2800 error value =  0.8187810045967847 W =  [[0.85075013]
 [0.08723009]] , b =  [-8.04546555]
step =  3200 error value =  0.764250750498313 W =  [[0.89143689]
 [0.10156861]] , b =

# XOR 문제

In [15]:
#손실함수, 수치미분함수 정의    

In [16]:
# LogicGate Class

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata):  # xdata, tdata => numpy.array(...)
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4,2)
        self.__tdata = tdata.reshape(4,1)
        
        # 가중치 W, 바이어스 b 초기화
        self.__W = np.random.rand(2,1)  # weight, 2 X 1 matrix
        self.__b = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
        
    # 손실함수
    def __loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )      
    
    # 손실 값 계산
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)
        
     # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )

    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.__loss_func()
        
        print("Initial error value = ", self.error_val())
        
        for step in  range(8001):
            
            self.__W -= self.__learning_rate * numerical_derivative(f, self.__W)
    
            self.__b -= self.__learning_rate * numerical_derivative(f, self.__b)
    
            if (step % 400 == 0):
                print("step = ", step, "error value = ", self.error_val())
                
                
    # 미래 값 예측 함수
    def predict(self, input_data):
        
        z = np.dot(input_data, self.__W) + self.__b
        y = sigmoid(z)
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

## AND

In [17]:
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,0,0,1]) #and 정답데이터

AND_obj=LogicGate("AND_GATE",xdata,tdata)

AND_obj.train()
# AND GATE prediction
print(AND_obj.name,"\n")

test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for input_data in test_data:
    (signoid_val,logical_val)=AND_obj.predict(input_data)
    print(input_data,"=",logical_val,"\n")

Initial error value =  5.356335015746674
step =  0 error value =  5.286377470844243
step =  400 error value =  1.509040088808328
step =  800 error value =  1.1282890527249734
step =  1200 error value =  0.9097813376496805
step =  1600 error value =  0.7646953923416763
step =  2000 error value =  0.6600581980109319
step =  2400 error value =  0.5805472909130298
step =  2800 error value =  0.5179086294968621
step =  3200 error value =  0.4672277644124635
step =  3600 error value =  0.4253637905647412
step =  4000 error value =  0.3902014230519888
step =  4400 error value =  0.3602576910916748
step =  4800 error value =  0.33445925815178623
step =  5200 error value =  0.31200882543490926
step =  5600 error value =  0.29230117119751464
step =  6000 error value =  0.2748683427739617
step =  6400 error value =  0.259342730089926
step =  6800 error value =  0.24543151227512697
step =  7200 error value =  0.23289856374974854
step =  7600 error value =  0.22155138382964296
step =  8000 error va

## OR

In [18]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata)

OR_obj.train() 
# OR Gate prediction
print(OR_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = OR_obj.predict(input_data) 
    print(input_data, " = ", logical_val, "\n") 

Initial error value =  1.8277152691656233
step =  0 error value =  1.8195223006536128
step =  400 error value =  1.035926582490207
step =  800 error value =  0.7622660402385871
step =  1200 error value =  0.5979771851631374
step =  1600 error value =  0.4892906157388449
step =  2000 error value =  0.4125053669868566
step =  2400 error value =  0.355625831624252
step =  2800 error value =  0.31194860655646944
step =  3200 error value =  0.27744528146801856
step =  3600 error value =  0.24955648348574766
step =  4000 error value =  0.22658264741089848
step =  4400 error value =  0.20735360158623664
step =  4800 error value =  0.19103894291045553
step =  5200 error value =  0.17703404810834109
step =  5600 error value =  0.1648888371717412
step =  6000 error value =  0.15426175385646376
step =  6400 error value =  0.14488917150836353
step =  6800 error value =  0.13656453675555572
step =  7200 error value =  0.1291238327902657
step =  7600 error value =  0.12243524533352472
step =  8000 e

## XOR 1

In [19]:
#XOR
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,1,1,0]) #xor 정답데이터

XOR_obj=LogicGate("XOR_GATE",xdata,tdata)

XOR_obj.train()

Initial error value =  3.2294859222851127
step =  0 error value =  3.2178934694146983
step =  400 error value =  2.7791509022906937
step =  800 error value =  2.7742033182404757
step =  1200 error value =  2.7730125964552492
step =  1600 error value =  2.7727037942396553
step =  2000 error value =  2.772620155825206
step =  2400 error value =  2.7725969759199187
step =  2800 error value =  2.7725904771734395
step =  3200 error value =  2.772588644868442
step =  3600 error value =  2.7725881268468
step =  4000 error value =  2.7725879802027293
step =  4400 error value =  2.7725879386641417
step =  4800 error value =  2.7725879268943694
step =  5200 error value =  2.772587923558985
step =  5600 error value =  2.7725879226137216
step =  6000 error value =  2.7725879223458207
step =  6400 error value =  2.7725879222698926
step =  6800 error value =  2.772587922248373
step =  7200 error value =  2.7725879222422734
step =  7600 error value =  2.772587922240545
step =  8000 error value =  2.7

In [20]:
# XOR GATE prediction -> 틀림
print(AND_obj.name,"\n")

test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for input_data in test_data:
    (signoid_val,logical_val)=XOR_obj.predict(input_data)
    print(input_data,"=",logical_val,"\n")

AND_GATE 

[0 0] = 0 

[0 1] = 0 

[1 0] = 0 

[1 1] = 1 



## NAND

In [30]:
#NAND
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata)

NAND_obj.train()
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for input_data in test_data:
    (sigmoid_val, logical_val) = NAND_obj.predict(input_data) 
    print(input_data, " = ", logical_val, "\n")  

Initial error value =  2.983874470255881
step =  0 error value =  2.973882490101613
step =  400 error value =  1.6249519584777996
step =  800 error value =  1.1870253563616546
step =  1200 error value =  0.9462210230593727
step =  1600 error value =  0.7899322146570983
step =  2000 error value =  0.6787300695103999
step =  2400 error value =  0.5949801076724933
step =  2800 error value =  0.5294197999961494
step =  3200 error value =  0.47662931098363515
step =  3600 error value =  0.4331878413030851
step =  4000 error value =  0.39681316517305726
step =  4400 error value =  0.3659169415645448
step =  4800 error value =  0.339356303570435
step =  5200 error value =  0.31628640026221666
step =  5600 error value =  0.29606851663996486
step =  6000 error value =  0.278210518643327
step =  6400 error value =  0.26232696552811463
step =  6800 error value =  0.2481116463079293
step =  7200 error value =  0.23531821781762155
step =  7600 error value =  0.22374627147607035
step =  8000 error v

## XOR Multi-Layer

In [36]:
# XOR 을 NAND + OR => AND 조합으로 계산함
input_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

s1 = []    # NAND 출력
s2 = []    # OR 출력

new_input_data = []  # AND 입력
final_output = []    # AND 출력

for index in range(len(input_data)):
    
    s1 = NAND_obj.predict(input_data[index])  # NAND 출력
    s2 = OR_obj.predict(input_data[index])    # OR 출력
    
    new_input_data.append(s1[-1])    # AND 입력
    new_input_data.append(s2[-1])    # AND 입력
    
    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))
    
    final_output.append(logical_val)    # AND 출력, 즉 XOR 출력    
    new_input_data = []    # AND 입력 초기화


for index in range(len(input_data)):    
    print(input_data[index], " = ", final_output[index], end='')
    print("\n")

[0 0]  =  0

[0 1]  =  1

[1 0]  =  1

[1 1]  =  0

